In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
import statistics 
from pandas import get_dummies
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,mean_squared_error
from sklearn.metrics import roc_auc_score,roc_curve
from imblearn.pipeline import Pipeline as imbPipe
from xgboost import  XGBClassifier
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df_train = pd.read_csv("../input/hr-analytics-job-change-of-data-scientists/aug_train.csv")
df_test = pd.read_csv("../input/hr-analytics-job-change-of-data-scientists/aug_test.csv")

In [ ]:
df_train

In [ ]:
df_train.info()

In [ ]:
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

# Visualization

## Bar plots
### showing the frequency of each category separated by label

In [ ]:
#Countplots showing the frequency of each category with respect to education level 
plt.figure(figsize=[15,17])
plot=["relevent_experience", "education_level","major_discipline", "experience","company_size","company_type", "training_hours","target"]
n=1
for f in plot:
    plt.subplot(4,2,n)
    sns.countplot(x=f, hue='education_level', edgecolor="black", alpha=0.7, data=df_train)
    sns.despine()
    plt.title("Countplot of {}  by education_level".format(f))
    n=n+1
plt.tight_layout()
plt.show()

## Education Level

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go

el = df_train['education_level'].value_counts().reset_index()
el.columns = [
    'education_level', 
    'percent'
]
el['percent'] /= len(df_train)

fig = px.pie(
    el, 
    names='education_level', 
    values='percent', 
    title='Education_level', 
    width=800,
    height=500 
)

fig.show()

In [ ]:
sns.pairplot(df_train)

## Experience

In [ ]:
# Enrolee total experience in years
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go

ep = df_train['experience'].value_counts().reset_index()
ep.columns = [
    'experience', 
    'percent'
]
ep['percent'] /= len(df_train)

fig = px.pie(
    ep, 
    names='experience', 
    values='percent', 
    title='Experience', 
    width=800,
    height=500 
)

fig.show()

In [ ]:
def clean_experience(df):
    for i in df["experience"]:
        if(i==">20"):
            df["experience"][df["experience"]==i]=27
        if(i == "<1"):
            df["experience"][df["experience"]==i]=0
clean_experience(df_train)
clean_experience(df_test)

df_train["experience"] = df_train["experience"].fillna(0)
df_train["experience"] = df_train['experience'].astype('int')
df_test["experience"] = df_test["experience"].fillna(0)
df_test["experience"] = df_test['experience'].astype('int')

## Training Hours

In [ ]:
f, axes = plt.subplots(1,1, figsize = (16, 5))
g1 = sns.distplot(df_train["training_hours"], color="blue",ax = axes)
plt.title("Distributional of training_hours")

## Education level to training hours

In [ ]:
# education_level:training_hours
et = df_train.sort_values(by='training_hours', ascending=True)[:7000]
figure = plt.figure(figsize=(10,6))
sns.barplot(y=et.education_level, x=et.training_hours)
plt.xticks()
plt.xlabel('training_hours')
plt.ylabel('education_level')
plt.title('education_level:training_hours ')
plt.show()

## Count :City Development Index

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go

cd = df_train['city_development_index'].value_counts().reset_index()
cd.columns = [
    'city_development_index', 
    'count'
]
cd['city_development_index'] = cd['city_development_index'].astype(str) + '-'
cd = cd.sort_values(['count']).tail(50)

fig = px.bar(
    cd, 
    x='count', 
    y='city_development_index', 
    orientation='h', 
    title='Count: City development index', 
    width=1000,
    height=900 
)

fig.show()

## City to City Development Index

In [ ]:
cdi = df_train.sort_values(by='city_development_index', ascending=True)[:2000]
figure = plt.figure(figsize=(10,6))
sns.barplot(y=cdi.city, x=cdi.city_development_index)
plt.xticks()
plt.xlabel('city_development_index')
plt.ylabel('city')
plt.title('City by city development index')
plt.show()

In [ ]:
f, axes = plt.subplots(1,1, figsize = (16, 5))
g1 = sns.distplot(df_train["city_development_index"], color="red",ax = axes)
plt.title("Distributional of city_development_index")

## Preprocessing

In [ ]:
def clean_NAN(df):
    df["gender"] = df["gender"].fillna("Unknown")
    df["education_level"]=df["education_level"].fillna("Unknown")
    df["major_discipline"].fillna(value="Unknown", inplace=True)
    df["experience"] = df["experience"].fillna(df["experience"].mean())
    df["company_type"] = df["company_type"].fillna("Unknown")
clean_NAN(df_train)
clean_NAN(df_test)

In [ ]:
def clean_company_size_1(df):
    converted_list_1 = []
    converted_list_2 = []
    converted_list_3 = []
    for i in df["company_size"]:
        if i == "10/49":
            i = "10-49"
            converted_list_1.append(i)
        converted_list_1.append(i)
        if i == "<10":
            i = '1-9'
            converted_list_2.append(i)
        converted_list_2.append(i)
        if i == "10000+":
            i = '10000-20000'
            converted_list_3.append(i)
        converted_list_3.append(i)
    df["company_size"]=pd.Series(converted_list_1)
    df["company_size"]=pd.Series(converted_list_2)
    df["company_size"]=pd.Series(converted_list_3)
    new = df['company_size'].str.split("-", n = 1, expand = True) 
    df['company_size_min']= new[0]
    df['company_size_max']= new[1] 
    df["company_size_max"] = df['company_size_max'].astype('int')
    df["company_size_min"] = df['company_size_min'].astype('int')
df_train["company_size"]=df_train["company_size"].fillna("0-0")
df_test["company_size"]=df_test["company_size"].fillna("0-0")
clean_company_size_1(df_train)
clean_company_size_1(df_test)

In [ ]:
def clean_last_new_job(df):
    converted_list_1 = []
    converted_list_2 = []
    for i in df["last_new_job"]:
        if i == "never" or i == np.NaN:
            i = 0
            converted_list_1.append(i)
        converted_list_1.append(i)
        if i == ">4":
            i = 6
            converted_list_2.append(i)
        converted_list_2.append(i)
    df["last_new_job"]=pd.Series(converted_list_1)
    df["last_new_job"]=pd.Series(converted_list_2)
clean_last_new_job(df_train)
clean_last_new_job(df_test)

In [ ]:
def clean_city(df):
    converted_list_1 = []
    for i in range(len(df["city"])):
        j = df["city"][i].replace("city_","")
        converted_list_1.append(j)
    df["city"]=pd.Series(converted_list_1)
clean_city(df_train)
clean_city(df_test)

In [ ]:
def clean_relevent_experience(df):
    converted_list_1 = []
    converted_list_2 = []
    for i in df["relevent_experience"]:
        if i == "Has relevent experience":
            i = 1
            converted_list_1.append(i)
        converted_list_1.append(i)
        if i == "No relevent experience":
            i = 0
            converted_list_2.append(i)
        converted_list_2.append(i)
    df["relevent_experience"]=pd.Series(converted_list_1)
    df["relevent_experience"]=pd.Series(converted_list_2)
clean_relevent_experience(df_train)
clean_relevent_experience(df_test)

In [ ]:
def one_hot_encoding(df):
    enrolled_dummies = pd.get_dummies(df["enrolled_university"], dummy_na=True)
    gender_dummies = pd.get_dummies(df["gender"], dummy_na=True)
    education_dummies = pd.get_dummies(df["education_level"],dummy_na=True)
    stream_dummies = pd.get_dummies(df["major_discipline"],dummy_na=True)
    company_dummies = pd.get_dummies(df["company_type"],dummy_na=True)
    df["Type_no_enrollment"] = enrolled_dummies["no_enrollment"]
    df["Type_Full_time_course"] = enrolled_dummies["Full time course"]
    df["Type_Part_time_course"]=enrolled_dummies["Part time course"]
    df["Gender_Male"] = gender_dummies["Male"]
    df["Gender_Female"] =gender_dummies["Female"]
    df["Gender_Unknown"]=gender_dummies["Unknown"]
    df["Gender_Other"]=gender_dummies["Other"]
    df["Education_Graduate"] = education_dummies["Graduate"]
    df["Education_Masters"] = education_dummies["Masters"]
    df["Education_High_School"] = education_dummies["High School"]
    df["Education_Primary_School"] = education_dummies["Primary School"]
    df["Education_Phd"] = education_dummies["Phd"]
    df["Education_Unknown"] = education_dummies["Unknown"]
    df["Stream_STEM"] = stream_dummies["STEM"]
    df["Stream_Humanities"] = stream_dummies["Humanities"]
    df["Stream_Other"] = stream_dummies["Other"]
    df["Stream_Business_Degree"] = stream_dummies["Business Degree"]
    df["Stream_Arts"] = stream_dummies["Arts"]
    df["Stream_No_Major"] = stream_dummies["No Major"]
    df["Stream_Unknown"] = stream_dummies["Unknown"]
    df["Company_Pvt_Ltd"] = company_dummies["Pvt Ltd"]
    df["Company_Funded_Startup"] = company_dummies["Funded Startup"]
    df["Company_Public_Sector"]=company_dummies["Public Sector"]
    df["Company_Early_Stage_Startup"] = company_dummies["Early Stage Startup"]
    df["Company_NGO"] = company_dummies["NGO"]
    df["Company_Other"] = company_dummies["Other"]
    df["Company_Unknown"] = company_dummies["Unknown"]
one_hot_encoding(df_train)
one_hot_encoding(df_test)


In [ ]:
df_train = df_train.dropna(subset=['enrolled_university',"last_new_job"])
df_test = df_test.dropna(subset=['enrolled_university',"last_new_job"])

In [ ]:
def clean_company_size_2(df):
    converted_list_1 = []
    converted_list_2 = []
    for i in df["company_size_min"]:
        if i == 0:
            i = int(df["company_size_min"].mean())
            converted_list_1.append(i)
        converted_list_1.append(i)
    for i in df["company_size_max"]:
        if i == 0:
            i = int(df["company_size_max"].mean())
            converted_list_2.append(i)
        converted_list_2.append(i)
    df["company_size_min"]=pd.Series(converted_list_1)
    df["company_size_max"]=pd.Series(converted_list_2)
df_train["company_size_min"] = df_train["company_size_min"].fillna(int(df_train["company_size_min"].mean()))
df_train["company_size_max"] = df_train["company_size_max"].fillna(int(df_train["company_size_max"].mean()))

df_test["company_size_min"] = df_test["company_size_min"].fillna(int(df_test["company_size_min"].mean()))
df_test["company_size_max"] = df_test["company_size_max"].fillna(int(df_test["company_size_max"].mean()))

clean_company_size_2(df_test)
clean_company_size_2(df_train)

In [ ]:
df_test.isnull().sum()

In [ ]:
# Target
# 0 – Not looking for job change,
# 1 – Looking for a job change
# As you can see, here we have imbalanced data, the number of 1 ( Looking for a job change) < 0 (Not looking for job change)
mnj = df_train['target'].value_counts()  
plt.figure(figsize=(6,4))
sns.barplot(mnj.index, mnj.values, alpha=0.8)
plt.ylabel('Number of Data', fontsize=12)
plt.xlabel('target', fontsize=9)
plt.xticks(rotation=90)
plt.show();

In [ ]:
df_test.index = np.arange(0,len(df_test))

In [ ]:
df_test_copy = df_test.copy()
df_test

In [ ]:
df_train = df_train.drop(['enrollee_id','gender','enrolled_university','education_level','major_discipline','company_type','company_size'],axis=1)
df_test = df_test.drop(['enrollee_id','gender','enrolled_university','education_level','major_discipline','company_type',"company_size"],axis=1)

In [ ]:
X = df_train.drop("target",axis=1)
Y = pd.DataFrame(df_train["target"])

## Data Upscalling

In [ ]:
smote = SMOTE()
X, Y = smote.fit_resample(X, Y)

In [ ]:
X

In [ ]:
Y["target"].value_counts()

In [ ]:
df_train_final = X.copy()
df_train_final['target'] = Y
df_test_final = df_test.copy()

In [ ]:
cols_to_be_normalized = ["city","city_development_index","experience","last_new_job","training_hours","company_size_min","company_size_max"]
cols_not_to_be_normalized = ["relevent_experience","Type_no_enrollment","Type_Full_time_course","Type_Part_time_course","Gender_Male","Gender_Female","Gender_Unknown",
                            "Gender_Other","Education_Graduate","Education_Masters","Education_High_School","Education_Primary_School","Education_Phd",
                            "Education_Unknown","Stream_STEM","Stream_Humanities","Stream_Other","Stream_Business_Degree","Stream_Arts","Stream_No_Major",
                            "Stream_Unknown","Company_Pvt_Ltd","Company_Funded_Startup","Company_Public_Sector", "Company_Early_Stage_Startup", "Company_NGO",
                            "Company_Other", "Company_Unknown", "target"]

## Normalisation

In [ ]:
train_normalized = normalize(df_train_final[cols_to_be_normalized])
train_boolean = df_train_final[cols_not_to_be_normalized]
df_train_normalized = pd.DataFrame(train_normalized,columns = cols_to_be_normalized)
df_train_boolean = pd.DataFrame(train_boolean,columns=cols_not_to_be_normalized)

In [ ]:
cols_to_be_normalized = ["city","city_development_index","experience","last_new_job","training_hours","company_size_min","company_size_max"]
cols_not_to_be_normalized = ["relevent_experience","Type_no_enrollment","Type_Full_time_course","Type_Part_time_course","Gender_Male","Gender_Female","Gender_Unknown",
                            "Gender_Other","Education_Graduate","Education_Masters","Education_High_School","Education_Primary_School","Education_Phd",
                            "Education_Unknown","Stream_STEM","Stream_Humanities","Stream_Other","Stream_Business_Degree","Stream_Arts","Stream_No_Major",
                            "Stream_Unknown","Company_Pvt_Ltd","Company_Funded_Startup","Company_Public_Sector", "Company_Early_Stage_Startup", "Company_NGO",
                            "Company_Other", "Company_Unknown"]

In [ ]:
test_normalized = normalize(df_test_final[cols_to_be_normalized])
test_boolean = df_test_final[cols_not_to_be_normalized]
df_test_normalized = pd.DataFrame(test_normalized,columns = cols_to_be_normalized)
df_test_boolean = pd.DataFrame(test_boolean,columns=cols_not_to_be_normalized)

In [ ]:
df_train_final = df_train_normalized.merge(df_train_boolean,left_index=True, right_index=True)
df_test_final = df_test_normalized.merge(df_test_boolean,left_index=True, right_index=True)
df_test_final.index = np.arange(0,len(df_test_final))
df_test_final

In [ ]:
df_test_final

## Splitting Independent and Dependent variables

In [ ]:
X = df_train_final.drop("target",axis = 1)
Y = df_train_final["target"]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42,shuffle=True, stratify = Y)

# Models

## Logistic Regression

In [ ]:
logitsic_model = LogisticRegression()

In [ ]:
logitsic_model.fit(X_train,Y_train)

In [ ]:
Y_pred = logitsic_model.predict(X_test)

In [ ]:
print(classification_report(Y_test,Y_pred))

In [ ]:
confusion_matrix(Y_test,Y_pred)

In [ ]:
print(mean_squared_error(Y_test,Y_pred))

In [ ]:
print ("Accuracy : ", accuracy_score(Y_test, Y_pred)) 

## Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
# Now we can try setting different learning rates, so that we can compare the performance of the classifier's 
#performance at different learning rates. Let's see what the performance was for different learning rates:

lr_list = [0.005, 0.0075, 0.01, 0.025, 0.05,0.1,0.25,0.5,1,0.88,0.9,1]

for learning_rate in lr_list:
    gb_clf = GradientBoostingClassifier(n_estimators=50, learning_rate=learning_rate, max_features=2, max_depth=2, random_state=0)
    gb_clf.fit(X_train, Y_train)

    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, Y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(X_test, Y_test)))

## XGBoost

In [ ]:
import warnings
warnings.filterwarnings("ignore")
XGBoost_pipe = imbPipe([
    ("XGBoost", XGBClassifier(random_state=42,n_jobs=-1,tree_method="hist"))
])

params={
    "XGBoost__max_depth": [20,21],
    "XGBoost__min_child_weight":[22,23],
    "XGBoost__n_estimators":[25,27],
    "XGBoost__subsample":[0.4,0.5,0.6],
    "XGBoost__colsample_bytree":[0.4,0.5,0.6],
    "XGBoost__gamma":[1,2,3],
    
}

XGBoost_grid = GridSearchCV(XGBoost_pipe, params, n_jobs=-1,cv=3,scoring="roc_auc")
XGBoost_grid.fit(X_train, Y_train)
print("Best Parameters for Model:  ",XGBoost_grid.best_params_)
Y_pred=XGBoost_grid.predict(X_train)
print("\n")
print(classification_report(Y_train, Y_pred))

## Predictions

In [ ]:
Y_pred=XGBoost_grid.predict(X_test)  
print(classification_report(Y_test, Y_pred))

In [ ]:
accuracy_score(Y_test,Y_pred)

In [ ]:
X_test = df_test_final.copy()

In [ ]:
# We predict Target values
Y_pred = XGBoost_grid.predict(X_test)  
Y_pred

In [ ]:
#Create Dataframe
submission = pd.DataFrame(df_test_copy['enrollee_id'])
submission["target"] = Y_pred

submission.head()

#### Convert DataFrame to a csv file that can be uploaded
#### This is saved in the same directory as your notebook

In [ ]:
filename = 'submission.csv'
submission.to_csv(filename,index=False)


print('Saved file: ' + filename)